# Database setting

In [ ]:
import mysql.connector
HOST = ""
USER = ""
PASSWORD = ""

In [ ]:


mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD
)

mycursor = mydb.cursor(dictionary=True)
try:
    mycursor.execute("CREATE DATABASE yonflix")
except:
    pass
mycursor.close()
mydb.close()

## 관련 테이블 생성

In [ ]:
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="yonflix"
)

mycursor = mydb.cursor(dictionary=True)

mycursor.execute("""CREATE TABLE `movie` (
  `id`    int(16) auto_increment PRIMARY KEY, 
  `story` text,
  `title` varchar(255) DEFAULT NULL,
  `link` varchar(255) DEFAULT NULL,
  `image` varchar(255) DEFAULT NULL
);
""")


In [ ]:
mycursor.execute("""CREATE TABLE `user` (
  `id`    int(16) auto_increment PRIMARY KEY, 
  `username` varchar(255) DEFAULT NULL
);
""")

In [ ]:
mycursor.execute("""CREATE TABLE `like` (
  `id`    int(16) auto_increment PRIMARY KEY, 
  `user_id` int(16),
  `movie_id` int(16),
   FOREIGN KEY (user_id) REFERENCES user (id),
   FOREIGN KEY (movie_id) REFERENCES movie (id)
);
""")

## 데이터 삽입

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("movie_doc.csv",sep="\t",index_col=0)

In [ ]:
df

In [ ]:
import requests
    
for row in df.to_dict(orient="records"):
    title  = row["title"]
    story  = row["story"]
    link   = row["link"]
    image  = row["image"]
    
    sql = "INSERT INTO movie (title, story,link,image) VALUES (%s, %s, %s, %s)"
    val = (title, story, link, image)
    mycursor.execute(sql, val)

mydb.commit()
    

In [ ]:
mycursor.execute("SELECT * FROM movie LIMIT 1")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)


In [ ]:
myresult

## 가상의 데이터로 확인

In [ ]:
for i in range(5):
    username = "user"+str(i)
    sql = "INSERT INTO user (username) VALUES (%s)"
    val = (username,)

    mycursor.execute(sql, val)
mydb.commit()

In [ ]:
mycursor.execute("SELECT * FROM user")

myresult = mycursor.fetchall()

for x in myresult:
    
    print(x)


In [ ]:


sql = "INSERT INTO `like` (user_id, movie_id) VALUES (%s,%s);"
val = [(1,1),(1,2),(1,3),(2,1),(3,4)]
mycursor.executemany(sql, val)

mydb.commit()


In [ ]:
mycursor.execute("SELECT * FROM `like`")

myresult = mycursor.fetchall()

for x in myresult:
    
    print(x)


In [ ]:
user = 2
movies = [1,2,3,4]

sql = "SELECT * FROM `like` WHERE `user_id` = %s"
val = (user,)
mycursor.execute(sql, val)

myresult = mycursor.fetchall()

for x in myresult:
    
    print(x)


In [ ]:
sql = "SELECT * FROM `like` WHERE 1"
mycursor.execute(sql)

likes = mycursor.fetchall()


In [ ]:
likes

In [ ]:
number_of_users = len(set([x['user_id'] for x in likes]))
number_of_movies = len(set([x['movie_id'] for x in likes]))

In [ ]:
import numpy as np
data = np.zeros((number_of_users,number_of_movies))

In [ ]:

for like in likes:
    data[like['user_id']-1,like['movie_id']-1] = 1

In [ ]:
users = sorted(set(["user"+str(x['user_id']) for x in likes]))

df = pd.DataFrame(data=data,index=users)
df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sims = cosine_similarity(df)
sims

In [ ]:
user=0
np.fill_diagonal(sims, 0)
themost_similar_user = np.argmax(sims[user])
themost_similar_user

In [ ]:
movie_diff = df.values[themost_similar_user] - df.values[user]
if movie_diff.sum() <= 0:
    print("You have watched all")
else:
    for k,v in enumerate(movie_diff):

        if v == 1:
            movie_id = k
            print("It's time to watch movie"+str(movie_id))